In [1]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

from otgnn.models import GCN
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model

from mol_opt.mol_opt import MolOpt
from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, get_latest_model
from mol_opt.ot_utils import compute_barycenter

from rdkit.Chem import MolFromSmiles

import torch
import numpy as np
import time
import torch.nn.functional as F

/home/octav/gitrepos/tum-thesis


In [2]:
model_type = "deepsets"
sys.argv = ["", "-cuda", "-model_type", model_type]
args = get_args()
outdir_suffix = "dev3"
args.n_epochs = 30 
args.init_model = "{}-chembl1".format(model_type)
# args.init_model = "{}-test1".format(model_type)
args.init_decoder_model = "{}_decode".format(args.init_model)
args.output_dir = "mol_opt/output_{}/{}".format(outdir_suffix, args.init_model)
args.tb_logs_dir = "mol_opt/logs_{}/{}".format(outdir_suffix, args.init_model)
args.batch_size = 500

args.penalty_gumbel = False 

args.n_hidden = 400

# for long runs
args.scale_lambdas = True
args.connectivity = True 
args.valency = True 
args.euler_characteristic_penalty = True
args.conn_lambda_start = 0.001
args.conn_lambda_end = 3
args.conn_lambda_epochs_start = 12
args.conn_lambda_epochs_end = 20
args.valency_lambda_start = 0.001
args.valency_lambda_end = 5
args.valency_lambda_epochs_start = 12
args.valency_lambda_epochs_end = 20
args.euler_lambda_start = 0.001 
args.euler_lambda_end = 2 
args.euler_lambda_epochs_start = 12
args.euler_lambda_epochs_end = 20
args.tau_start = 1
args.tau_end = 0.01
args.tau_epochs_start = 1
args.tau_epochs_end = 17

args.morgan_bits = 0

args.conn_penalty_function = "capped_logdet" 

print(args, '\n')

Namespace(N_transformer=6, agg_func='sum', batch_norm=False, batch_size=500, conn_lambda_end=3, conn_lambda_epochs_end=20, conn_lambda_epochs_start=12, conn_lambda_start=0.001, conn_penalty_function='capped_logdet', connectivity=True, connectivity_hard=False, cross_att_dim=150, cross_att_use=False, cuda=True, device='cuda:0', dim_tangent_space=40, dropout_ffn=0.0, dropout_gcn=0.0, dropout_transformer=0.1, euler_characteristic_penalty=True, euler_lambda_end=2, euler_lambda_epochs_end=20, euler_lambda_epochs_start=12, euler_lambda_start=0.001, ffn_activation='LeakyReLU', init_decoder_model='deepsets-chembl1_decode', init_model='deepsets-chembl1', linear_out=False, max_num_atoms=70, model_type='deepsets', morgan_bits=0, n_epochs=30, n_ffn_hidden=100, n_ffn_transformer=100, n_heads_transformer=10, n_hidden=400, n_labels=1, n_layers=5, one_batch_train=False, ot_solver='emd', output_dir='mol_opt/output_dev3/deepsets-chembl1', pc_hidden=150, penalty_gumbel=False, pred_hidden=150, rec_lambda_e

In [3]:
# model_type = "molemb"
# sys.argv = ["", "-cuda", "-model_type", model_type, "-one_batch_train"]
# args = get_args()
# outdir_suffix = "dev3"
# args.n_epochs = 15000 
# args.init_model = "{}-morgan-test1".format(model_type)
# # args.init_model = "{}-test1".format(model_type)
# args.init_decoder_model = "{}_decode".format(args.init_model)
# args.output_dir = "mol_opt/output_{}/{}".format(outdir_suffix, args.init_model)
# args.tb_logs_dir = "mol_opt/logs_{}/{}".format(outdir_suffix, args.init_model)
# args.batch_size = 50 

# args.penalty_gumbel = False 

# args.scale_lambdas = True
# args.connectivity = True 
# args.valency = True 
# args.euler_characteristic_penalty = True
# args.conn_lambda_start = 0.001
# args.conn_lambda_end = 3
# args.conn_lambda_epochs_start = 4001
# args.conn_lambda_epochs_end = 12000
# args.valency_lambda_start = 0.001
# args.valency_lambda_end = 5
# args.valency_lambda_epochs_start = 4001
# args.valency_lambda_epochs_end = 12000
# args.euler_lambda_start = 0.001 
# args.euler_lambda_end = 2 
# args.euler_lambda_epochs_start = 4001
# args.euler_lambda_epochs_end = 12000
# args.tau_start = 1
# args.tau_end = 0.01
# args.tau_epochs_start = 1
# args.tau_epochs_end = 12000

# args.morgan_bits = 1024

# args.conn_penalty_function = "capped_logdet" 

# print(args, '\n')

In [4]:
# train_data_loader = get_loader("iclr19-graph2graph/data/qed", "train_pairs", args.batch_size, same_number_atoms = True)
# train_data_loader = get_loader("molgen/data/chembl50", "train", args.batch_size, same_number_atoms = True)
# val_data_loader = get_loader("iclr19-graph2graph/data/qed", "valid", 36, False)

train_data_loader = get_loader("molgen/data/chembl50", "train_split", args.batch_size, same_number_atoms = True)
val_data_loader = get_loader("molgen/data/chembl50", "val_split", args.batch_size, same_number_atoms = True)
# val_data_loader = None

In [5]:
molopt, molopt_decoder = main(args, train_data_loader = train_data_loader, val_data_loader = val_data_loader)

Found previous model mol_opt/output_dev3/deepsets-chembl1/model_deepsets-chembl1_20, epoch 20. Overwriting args.
Penalty params: tau=0.01000 conn_l=3.00000 val_l=5.00000 euler_l=2.00000 epoch=20 mode=[0 0 0] conn=True val=True euler=True
Epoch: 21


/home/octav/gitrepos/tum-thesis/otgnn/models/gromov_modules.py:398: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  nce_reg = torch.nn.LogSoftmax()(torch.stack(all_nce_dists))[0]


Losses Batch 0, train
 fgw:1.1903840
 conn_penalty:0.7469680
 val_penalty:0.5352757
 euler_penalty:3.7419802
 total:13.5916270
Penalty params: tau=0.01000 conn_l=3.00000 val_l=5.00000 euler_l=2.00000 epoch=21 mode=[0 0 0] conn=True val=True euler=True
Losses Batch 30, train
 fgw:1.1870097
 conn_penalty:1.3145099
 val_penalty:0.4423656
 euler_penalty:3.6304339
 total:14.6032352
Penalty params: tau=0.01000 conn_l=3.00000 val_l=5.00000 euler_l=2.00000 epoch=21 mode=[0 0 0] conn=True val=True euler=True
Losses Batch 60, train
 fgw:1.1444648
 conn_penalty:1.1738984
 val_penalty:0.3440101
 euler_penalty:3.3563245
 total:13.0988594
Penalty params: tau=0.01000 conn_l=3.00000 val_l=5.00000 euler_l=2.00000 epoch=21 mode=[0 0 0] conn=True val=True euler=True
Losses Batch 90, train
 fgw:1.1147998
 conn_penalty:1.0513408
 val_penalty:0.2746484
 euler_penalty:2.9775109
 total:11.5970863
Penalty params: tau=0.01000 conn_l=3.00000 val_l=5.00000 euler_l=2.00000 epoch=21 mode=[0 0 0] conn=True val=True 

Losses Batch 180, train
 fgw:1.0309995
 conn_penalty:0.5760163
 val_penalty:0.1679201
 euler_penalty:1.9240298
 total:7.4467086
Penalty params: tau=0.01000 conn_l=3.00000 val_l=5.00000 euler_l=2.00000 epoch=22 mode=[0 0 0] conn=True val=True euler=True
Losses Batch 210, train
 fgw:1.0208259
 conn_penalty:0.5518266
 val_penalty:0.1534692
 euler_penalty:1.8285105
 total:7.1006727
Penalty params: tau=0.01000 conn_l=3.00000 val_l=5.00000 euler_l=2.00000 epoch=22 mode=[0 0 0] conn=True val=True euler=True
Losses Batch 240, train
 fgw:1.0135193
 conn_penalty:0.5279674
 val_penalty:0.1437293
 euler_penalty:1.7604831
 total:6.8370341
Penalty params: tau=0.01000 conn_l=3.00000 val_l=5.00000 euler_l=2.00000 epoch=22 mode=[0 0 0] conn=True val=True euler=True
Losses Batch 270, train
 fgw:1.0075917
 conn_penalty:0.4982947
 val_penalty:0.1370229
 euler_penalty:1.6854112
 total:6.5584129
Penalty params: tau=0.01000 conn_l=3.00000 val_l=5.00000 euler_l=2.00000 epoch=22 mode=[0 0 0] conn=True val=True

KeyboardInterrupt: 

In [ ]:
molopt, molopt_decoder = main(args, train_data_loader = train_data_loader, val_data_loader = None)

In [ ]:
for i in train_data_loader:
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    break
    
x_embedding = molopt.forward(X)
yhat_logits = molopt_decoder.forward(x_embedding, X, Y)
yhat_labels = molopt_decoder.discretize_argmax(*yhat_logits)
# yhat_labels = molopt_decoder.discretize(*yhat_logits)
pred_pack = (yhat_labels, yhat_logits, Y.scope), Y 

target = Y.get_graph_outputs()
symbols_labels, charges_labels, bonds_labels = yhat_labels
symbols_logits, charges_logits, bonds_logits = yhat_logits

from mol_opt.ot_utils import FGW 
fgw_loss = FGW(alpha = 0.5)
fgw_loss(*pred_pack, tau = 1)

In [ ]:
torch.optim.Adam([torch.autograd.Variable(torch.Tensor([0.]))]).param_groups[0]['lr']

In [ ]:
0.9 ** (1/5)

In [ ]:
vars(args)

In [ ]:
from tensorboardX import SummaryWriter
writer = SummaryWriter("/home/octav/gitrepos/tum-thesis/mol_opt/dev7")

In [ ]:
tags = [
    "pointwise10-dev8/train_avg_euler_error/mean",
    "pointwise10-dev8/train_avg_euler_error/bot_band",
    "pointwise10-dev8/train_avg_euler_error/top_band"
]
# writer.add_custom_scalars_marginchart(tags)


In [ ]:
# writer.add_scalar("plm2/plm2", 0, 1)
# writer.add_scalar("plm2/plm2", 1, 2)
# writer.add_scalar("plm2/plm2", 0, 3)
# writer.add_scalar("plm2/plm2", 1, 4)
tags = ["plm", "plm", "plm"]
# layout = {"plm" : {"plm" : ["Margin", tags]}}
layout = {"plm1" : {"plm" : ["Margin", tags], "plm2": ["Multiline", tags]},
          "plm2": {"plm" : ["Margin", tags], "plm2": ["Multiline", tags]}}
writer.add_custom_scalars(layout)

In [ ]:
writer._get_file_writer().add_summary()

In [ ]:
writer.logdir

In [ ]:
writer.close()

In [ ]:
import tensorboardX
tensorboardX.__version__

In [ ]:
from tensorboardX.summary import custom_scalars

In [ ]:
writer._get_file_writer().add_summary(custom_scalars(layout), global_step = 3)
writer.flush()

In [ ]:
writer.add_scalars('plm', {'plm0' : 0.0, 'plmt' : 0.1, 'plmb' : -0.1}, 3)
writer.flush()

In [ ]:
tau = 1
symbols_nll, charges_nll, bonds_nll = F.gumbel_softmax(tau = tau, dim=1, logits = symbols_logits), F.gumbel_softmax(tau=tau,dim=1,logits=charges_logits), F.gumbel_softmax(tau=tau,dim=1, logits = bonds_logits)

In [ ]:
np.sort(symbols_nll.mean(axis = 0).detach().numpy())

In [ ]:
list(molopt.opt0.parameters())[0]

In [ ]:
molopt.state_dict()

In [ ]:
model_dict = torch.load('mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch_8')

In [ ]:
model_dict["model"]['opt0.weight']

In [ ]:
molopt2 = MolOpt(args)

In [ ]:
list(molopt2.opt0.parameters())[0]

In [ ]:
molopt2.load_state_dict(model_dict["model"])

In [ ]:
list(molopt2.opt0.parameters())[0]

In [ ]:
molopt3,args3 = load_model('mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch_8', MolOpt, args.device)

In [ ]:
list(molopt3.opt0.parameters())[0]

In [ ]:
fgw_loss(*pred_pack, tau = 1)